# Práctica: detección de fraude mediante métodos de ensembles

ESTUDIANTES: `ESCRIBID VUESTROS NOMBRES AQUI`

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fraud.jpg" style="width:600px;">

En esta práctica vamos a utilizar todos los conocimientos adquiridos en los ejercicios anteriores, con el objetivo de construir un detector automático de fraude en pagos con tarjeta.

## Instrucciones

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.
    <b>La nota máxima que puede obtenerse con esta clase de ejercicios es de 7 sobre 10.</b>
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema, y a conseguir una calificación más alta. <b>Resolviendo esta clase de ejercicios puedes llegar conseguir hasta 3 puntos sobre 10.</b> ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Ensembles recomendados](https://github.com/albarji/teaching-environments-ensembles).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [132]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

## Carga y preparación de datos

Para este problema usaremos una versión reducida de los datos disponibles en la competición de Kaggle [IEEE-CIS Fraud Detection](https://www.kaggle.com/c/ieee-fraud-detection/data). Los datos a emplear están incluidos en la carpeta *data*, con ficheros separados para entrenamiento y test. Cada fichero incluye una gran cantidad de variables explicativas sobre la naturaleza de la operación con tarjeta realizada: puede encontrarse información sobre la naturaleza de estas variables en las [discusiones de la competición](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203). La variable objetivo es `isFraud`, la cual toma el valor 1 para operaciones fraudulentas.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Carga los datos de entrenamiento y test en sendos Pandas DataFrames con nombres <b>train</b> y <b>test</b>, respectivamente.
</font>

***

In [150]:
# Leyendo datos
train = pd.read_csv('data/train.csv')

# Mostrando 5 valores al azar
train.sample(5)

,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
8218,109485,3096485,0,2151047,226.000,W,15320,555.0,150.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51051,221812,3208812,0,5233684,50.000,R,6019,583.0,150.0,visa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51057,292819,3279819,0,7233678,107.950,W,10057,225.0,150.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35388,585869,3572869,0,15685742,146.000,W,11162,346.0,150.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4737,405716,3392716,0,10254880,32.385,C,15885,545.0,185.0,visa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
# Leyendo datos
test = pd.read_csv('data/test.csv')

# Mostrando 5 valores al azar
test.sample(5)

,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
51191,283669,3270669,0,6925859,107.950,W,7664,490.0,150.0,visa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45759,98893,3085893,0,1989711,32.356,C,17039,500.0,185.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,560262,3547262,0,14843991,30.950,W,7166,435.0,150.0,visa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32935,238472,3225472,0,5619479,30.950,W,7815,161.0,150.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14165,293041,3280041,0,7237937,50.000,W,9749,181.0,150.0,visa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Explorando valores de las variables
for col, values in train.items():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

Unnamed: 0: 60000
[229713 179605 456372 ... 294899 401948  39309]


TransactionID: 60000
[3216713 3166605 3443372 ... 3281899 3388948 3026309]


isFraud: 2
[0 1]


TransactionDT: 59809
[ 5439892  3950157 11664039 ...  7274574 10134899   956013]


TransactionAmt: 5792
[ 49.     25.95  500.    ...  59.755  20.603  43.46 ]


ProductCD: 5
['W' 'R' 'C' 'S' 'H']


card1: 6059
[ 1556 18385  9849 ... 17831 12478 13284]


card2: 500
[314. 555. 277.  nan 399. 512. 248. 490. 361. 179. 321. 250. 111. 360.
 296. 225. 185. 476. 514. 396. 524. 100. 500. 491. 122. 199. 567. 438.
 410. 206. 180. 302. 489. 553. 583. 310. 496. 170. 540. 532. 421. 174.
 545. 194. 158. 130. 405. 420. 327. 509. 404. 562. 543. 103. 176. 320.
 519. 181. 215. 595. 132. 459. 589. 494. 340. 177. 305. 115. 142. 445.
 285. 453. 417. 548. 452. 375. 408. 393. 318. 161. 369. 585. 387. 298.
 238. 269. 332. 486. 430. 402. 556. 455. 143. 324. 247. 537. 134. 268.
 390. 257. 549. 237. 504. 213. 264. 216. 322. 373. 523. 286. 168. 347.
 204

Los datos a utilizar presentan algunas problemáticas que debes resolver antes de pasar a la construcción de modelos:
* Valores de variables expresados como strings. scikit-learn no acepta variables expresadas de esta forma.
* Valores faltantes en muchas variables. Aunque varios modelos de scikit-learn pueden trabajar con valores faltantes sin problemas, otros modelos fallarán al encontrarlos.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Aplica las transformaciones que creas necesarias para tener los datos listos para el modelado. Ten en cuenta que cualquier proceso realizado debe aplicarse tanto a los datos de train como a los de test.
</font>

***

In [101]:
# ELiminado variables que no aportan información
train = train.drop(columns = ['Unnamed: 0', 'TransactionID'], axis = 1)
test = test.drop(columns = ['Unnamed: 0', 'TransactionID'], axis = 1)

In [102]:
# Identificando columnas con muchos valores nulos
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [104]:
# Identificando columnas con un valor dominante excesivo (más del 90% de las filas comparten el mismo valor)
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [107]:
# Identificando columnas con un solo valor 
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]

In [110]:
# Identificando número de columnas a eliminar
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
len(cols_to_drop)

52

In [112]:
# Eliminando columnas en train y test
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [136]:
# Seleccionando columnas categóricas 
cat_cols = ['ProductCD',  
            'card1', 'card2', 'card3',  'card4', 'card5', 'card6', 
            'addr1', 'addr2',
            'P_emaildomain', 'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 
            'R_emaildomain', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3',
            'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']

# Convirtiendo variables categóricas a 'one-hot'
for col in cat_cols:
    if col in train.columns:
        encoder = OneHotEncoder(sparse_output=False)
        # Asegurarse de que los datos tienen la forma correcta (n_filas, 1)
        values = list(train[col].astype(str).values) + list(test[col].astype(str).values)
        reshaped_values = np.array(values).reshape(-1, 1)  # Convertir a 2D
        encoder.fit(reshaped_values)
        
        train[col] = encoder.transform(np.array(train[col].astype(str).values).reshape(-1, 1))
        test[col] = encoder.transform(np.array(test[col].astype(str).values).reshape(-1, 1))

In [156]:
# Ordenando por fecha de transanción y eliminando columna 
train = train.sort_values('TransactionDT').drop(['TransactionDT'], axis=1)
test = test.sort_values('TransactionDT').drop(['TransactionDT'], axis=1)

## Midiendo el rendimiento de un detector de fraude

Las actividades fraudulentas son constantemente perseguidas, por lo que los defraudadores necesitan ser creativos e inventar nuevas formas de llevar a cabo sus fraudes. Además, afortunadamente, las operaciones fraudulentas son relativamente escasas, lo que nos lleva a contar con pocos casos positivos para entrenar el modelo. Dicho de otro modo, nos enfrentamos a un problema altamente desequilibrado, lo cual dificulta el entrenamiento del modelo, así como su evaluación.

Consideremos un modelo trivial que clasifica todos los casos como negativos (operaciones legítimas). Podemos simular las predicciones de este modelo creando un vector de predicciones de todo ceros:

In [144]:
dumbpreds = [0] * len(test)

Midamos el porcentaje de acierto de este modelo sobre el conjunto de test

In [146]:
from sklearn.metrics import accuracy_score

accuracy_score(test["isFraud"], dumbpreds)

0.8333333333333334

Deberías haber obtenido en torno a un 83% de acierto, ya que la gran mayoría de casos son negativos. A pesar de eso, ¡este modelo es totalmente inútil como detector de fraudes! Por tanto, necesitamos una métrica mejor.

Una métrica que funciona bien para problemas muy desbalanceados es el [área bajo la curva ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic), o AUC. En scikit-learn esta métrica está disponible, y podemos probarla para comprobar que el rendimiento de este modelo es realmente malo

In [148]:
from sklearn.metrics import roc_auc_score

roc_auc_score(test["isFraud"], dumbpreds)

0.5

Un AUC del 50% indica que el modelo no es mejor que lanzar predicciones aleatoriamente. Si evaluáramos un modelo en el que las predicciones de probabilidad de la clase fraude fueran algo más altas para los casos realmente fraudulentos que para los casos legítimos, veríamos cómo el AUC produce mayores valores. El caso óptimo para esta métrica es un modelo en el que todos los casos de fraude son predichos con una mayor probabilidad de fraude que todos los casos de operaciones legítimas.

## Detector de fraude no supervisado

Dado que apenas tenemos datos de fraude, puede tener sentido empezar construyendo un modelo no supervisado.


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Usando <b>solo los datos de train</b>, y sin emplear la variable <i>Class</i>, construye un modelo de tipo IsolationForest para detección de anomalías. Mide el rendimiento del modelo sobre el conjunto de test, usando la métrica AUC.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Ten en cuenta los siguientes puntos para construir un buen modelo:
    <ul>
        <li>Puedes estimar el parámetro <i>contamination</i> como la proporción de datos fraudulentos del conjunto de entrenamiento.</li>
        <li>Por defecto IsolationForest emplea pocos árboles. Asegúrate de probar con diferentes números de árboles.</li>
        <li>La métrica AUC necesita recibir las <b>probabilidades de clase fraude</b> para funcionar correctamente. No es posible obtener probabilidades de un IsolationForest, pero puedes hacer uso de su función <i>score_samples</i> para obtener scores (profundidades medias en el árbol), los cuales pueden ser negados para obtener valores que pueden interpretarse como probabilidades de clase fraude (no normalizadas).</li>
    </ul>
    Si has entrenado el modelo correctamente, deberías obtener en torno a un <b>94% de AUC</b>.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea una visualización mostrando el rendimiento de este modelo sobre los datos de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Sugerencia: utiliza la <a href=https://scikit-learn.org/stable/modules/generated/sklearn.metrics.RocCurveDisplay.html>visualización ROC que se presenta en la documentación de scikit-learn</a>.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Árbol de detección de fraude

Ahora vamos a comprobar si un método supervisado sencillo e interpretable, como es un árbol de decisión, puede ayudarnos a resolver este problema.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Entrena un árbol de decisión sobre los datos de train, haciendo uso tanto de las variables explicativas como de la variable objetivo. Trata de que el árbol sea sencillo, de forma que puedas generar una visualización interpretable del mismo. Mide también los resultados del árbol en AUC sobre el conjunto de test. ¿Obtienes mejoras sobre el modelo no supervisado?
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Modelos supervisados de ensemble

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Vamos a comprobar cómo es posible mejorar los resultados de detección usando métodos de ensemble. Entrena <b>al menos 5 de los métodos de ensemble vistos en clase</b>, y mide sus resultados de AUC en el test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Algunas sugerencias de modelos disponibles en scikit-learn que puedes utilizar:
    <ul>
        <li>Random Forest</li>
        <li>Extra Trees</li>
        <li>AdaBoost</li>
        <li>Gradient Boosting</li>
        <li>Histogram-based Gradient Boosting</li>
        <li>Bagging</li>
        <li>Voting</li>
        <li>Stacking</li>
    </ul>
No olvides que para medir correctamente el AUC, debes suministrar a esta métrica las <b>probabilidades de clase fraude</b> predichas por el modelo, las cuales puedes obtener mediante el método <i>predict_proba</i>.
    
Si has entrenado los modelos correctamente, tu mejor resultado debería ser de al menos un <b>98% de AUC</b> en test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Adicionalmente, se valorará:
    <ul>
        <li>Utilizar más técnicas de ensembles, además de las 5 mínimas requeridas.</li>
        <li>Mejorar el AUC en test de tu modelo lo máximo posible.</li>
        <li>Utilizar <a href=https://catboost.ai/>CatBoost</a>, <a href=https://lightgbm.readthedocs.io/en/latest/>LightGBM</a> o <a href=https://auto.gluon.ai/>AutoGluon</a>, otras librerías de ensemble muy efectivas. Nótese que deberás instalar estas librerías en el entorno.</li>
    </ul>
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Visualización

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea una visualización mostrando el rendimiento de todos tus modelos supervisados sobre el conjunto de test, junto con el modelo no supervisado. La visualización debe mostrar claramente cuál es el modelo con mayor AUC.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE